In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

In [2]:
adata = sc.read_h5ad('../data/rna_figure_ready.h5ad')

In [3]:
sc.pp.highly_variable_genes(adata, n_top_genes=None)  # No filtering yet

In [4]:
threshold_50 = adata.var['dispersions_norm'].quantile(0.50)
adata.var['highly_variable'] = adata.var['dispersions_norm'] > threshold_50
hvg = adata.var_names[adata.var['highly_variable']]
len(hvg)

10701

In [5]:
tf_df = pd.read_csv('../data/uniprotkb_organism_id_9606_AND_go_00037_2025_01_29.tsv', sep='\t')
tf_list = tf_df['Gene Names'].str.split().explode().tolist()
len(tf_list)

3044

In [6]:
adata.var['is_tf'] = adata.var_names.isin(tf_list)
adata.var['is_tf'].sum()

1281

In [7]:
threshold = adata[:,adata.var['is_tf']].var['dispersions_norm'].quantile(0.50)
adata.var['is_tf_hvg'] = adata.var['is_tf'] & (adata.var['dispersions_norm'] > threshold)
hv_tf = adata.var_names[adata.var['is_tf_hvg']]
len(hv_tf)

640

In [8]:
print(f"Intersection length: {len(set(hvg).intersection(hv_tf))}")
print(f"HVG length: {len(hvg)}, HV TF length: {len(hv_tf)}")

Intersection length: 640
HVG length: 10701, HV TF length: 640


In [9]:
# Save HVGs and HV TFs to files
with open('../data/hvg_50.txt', 'w') as f:
    f.write('\n'.join(hvg))
    
with open('../data/hv_tf_50.txt', 'w') as f:
    f.write('\n'.join(hv_tf))


In [10]:
assert False

AssertionError: 

In [11]:
tf_hvg_intersection = set(tf_list).intersection(hvg)
print(f"Number of TFs: {len(tf_list)}")
print(f"Number of HVGs: {len(hvg)}")
print(f"Number of TFs in HVGs: {len(tf_hvg_intersection)}")

Number of TFs: 3044
Number of HVGs: 10701
Number of TFs in HVGs: 655


In [12]:
# sc.pp.highly_variable_genes(adata, n_top_genes=400)
alex_genes = open('../data/alex_genes.txt').read().splitlines()
alex_tfs = open('../data/alex_tfs.txt').read().splitlines()

In [13]:
alex_genes_in_hvg = [g for g in alex_genes if g in hvg]
alex_tfs_in_hvg = [t for t in alex_tfs if t in hvg]
print(f"Number of Alex genes in HVG: {len(alex_genes_in_hvg)}, total number of Alex genes: {len(alex_genes)}")
print(f"Number of Alex TFS in HVG: {len(alex_tfs_in_hvg)}, total number of Alex TFS: {len(alex_tfs)}")

Number of Alex genes in HVG: 1809, total number of Alex genes: 1970
Number of Alex TFS in HVG: 284, total number of Alex TFS: 331


In [14]:
tf_hvg_intersection = set(tf_list).intersection(hvg)
alex_tf_set = set(alex_tfs)
common_tfs = tf_hvg_intersection.intersection(alex_tf_set)

print(f"Number of TFs in HVGs: {len(tf_hvg_intersection)}")
print(f"Number of Alex TFs: {len(alex_tf_set)}")
print(f"Number of TFs in both sets: {len(common_tfs)}")


Number of TFs in HVGs: 655
Number of Alex TFs: 331
Number of TFs in both sets: 234


In [15]:
'ESRRA' in tf_hvg_intersection

False

In [16]:
'ESRRA' in hvg

False